# Defining MDM

In [1]:
def mdm_test(actual_lst, pred1_lst, pred2_lst, h = 1, crit="RMSE"):
    def error_check():
        rt = 0
        msg = ""
        if (not isinstance(h, int)):
            rt = -1
            msg = "The type of the number of steps ahead (h) is not an integer."
            return (rt,msg)
        if (h < 1):
            rt = -1
            msg = "The number of steps ahead (h) is not large enough."
            return (rt,msg)
        len_act = len(actual_lst)
        len_p1  = len(pred1_lst)
        len_p2  = len(pred2_lst)
        if (len_act != len_p1 or len_p1 != len_p2 or len_act != len_p2):
            rt = -1
            msg = "Lengths of actual_lst, pred1_lst and pred2_lst do not match."
            return (rt,msg)
        if (h >= len_act):
            rt = -1
            msg = "The number of steps ahead is too large."
            return (rt,msg)

        if (crit != "RMSE" and crit != "MAE" and crit != "MAPE"):
            rt = -1
            msg = "The criterion is not supported."
            return (rt,msg)  
        from re import compile as re_compile
        comp = re_compile("^\d+?\.\d+?$")  
        def compiled_regex(s):
            if comp.match(s) is None:
                return s.isdigit()
            return True
        for actual, pred1, pred2 in zip(actual_lst, pred1_lst, pred2_lst):
            is_actual_ok = compiled_regex(str(abs(actual)))
            is_pred1_ok = compiled_regex(str(abs(pred1)))
            is_pred2_ok = compiled_regex(str(abs(pred2)))
            if (not (is_actual_ok and is_pred1_ok and is_pred2_ok)):  
                msg = "An element in the actual_lst, pred1_lst or pred2_lst is not numeric."
                rt = -1
                return (rt,msg)
        return (rt,msg)
    

    error_code = error_check()

    if (error_code[0] == -1):
        raise SyntaxError(error_code[1])
        return

    from scipy.stats import t
    import collections
    import pandas as pd
    import numpy as np
    
    e1_lst = []
    e2_lst = []
    d_lst  = []
    
    actual_lst = pd.Series(actual_lst).apply(lambda x: float(x)).tolist()
    pred1_lst = pd.Series(pred1_lst).apply(lambda x: float(x)).tolist()
    pred2_lst = pd.Series(pred2_lst).apply(lambda x: float(x)).tolist()
    
    T = float(len(actual_lst))
    
    if crit == "RMSE":
        for actual, p1, p2 in zip(actual_lst, pred1_lst, pred2_lst):
            e1_lst.append((actual - p1) ** 2)
            e2_lst.append((actual - p2) ** 2)
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)

    if (crit == "MAE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs(actual - p1))
            e2_lst.append(abs(actual - p2))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)

    if (crit == "MAPE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs((actual - p1)/actual))
            e2_lst.append(abs((actual - p2)/actual))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
   
    mean_d = pd.Series(d_lst).mean()
    
    def autocovariance(Xi, N, k, Xs):
        autoCov = 0
        T = float(N)
        for i in np.arange(0, N-k):
              autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
        return (1/(T))*autoCov
    gamma = []
    for lag in range(0,h):
        gamma.append(autocovariance(d_lst,len(d_lst),lag,mean_d)) # 0, 1, 2
    V_d = (gamma[0] + 2*sum(gamma[1:]))/T
    MDM_stat=V_d**(-0.5)*mean_d
    harvey_adj=((T+1-2*h+h*(h-1)/T)/T)**(0.5)
    MDM_stat = harvey_adj*MDM_stat
    p_value = 2*t.cdf(-abs(MDM_stat), df = T - 1)
    
    dm_return = collections.namedtuple('mdm_return', 'MDM p_value')
    
    rt = dm_return(MDM = MDM_stat, p_value = p_value)
    
    return rt

# Importing real values

In [2]:
import random
import numpy as np
import pandas as pd
from pandas import read_csv
actual_lst = read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,22])
actual_lst=actual_lst.values
actual_lst=actual_lst.flatten()
actual_lst

array([3328.100204, 3332.149794, 3320.879784, 3326.04989 , 3264.449921,
       3258.34004 , 3275.869972, 3289.770123, 3279.860082, 3242.800176,
       3278.380004, 3286.620016, 3320.410089, 3326.460106, 3380.820085,
       3387.619927, 3365.840095, 3350.279892, 3327.229885, 3346.970149,
       3347.700019, 3358.889827, 3367.33015 , 3380.150078, 3431.650056,
       3413.929791, 3423.489862, 3371.449964, 3408.309944, 3456.740015,
       3457.639881, 3465.729852, 3452.160099, 3448.399848, 3449.580227,
       3428.659968, 3422.54011 , 3384.890013, 3383.180076, 3371.129927,
       3373.560141, 3378.660023, 3406.149881, 3413.810064, 3423.609972,
       3415.749861, 3394.209865, 3394.070184, 3397.010003, 3412.519936,
       3407.09004 , 3414.45014 , 3474.919999, 3511.659868, 3528.679812,
       3556.800152, 3576.20004 , 3588.060239, 3597.700128, 3608.340039,
       3622.349939, 3599.060096, 3589.270165, 3608.770209, 3603.149979,
       3589.960127, 3636.240113, 3616.540142, 3637.10007 , 3610.

# Import the predictions of the proposed model

In [3]:
pred1_lst =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,23])
pred1_lst =pred1_lst.values
pred1_lst=pred1_lst.flatten()
pred1_lst

array([3345.8625, 3319.0515, 3324.624 , 3319.9434, 3281.467 , 3259.2983,
       3276.3489, 3291.6667, 3283.19  , 3261.6187, 3267.6892, 3294.212 ,
       3309.8472, 3327.136 , 3361.7065, 3385.1345, 3372.2366, 3345.5898,
       3332.5405, 3339.1797, 3355.5227, 3357.1511, 3359.395 , 3383.1292,
       3409.4858, 3425.549 , 3413.53  , 3388.9575, 3399.9722, 3443.7715,
       3463.7495, 3458.358 , 3453.329 , 3448.7866, 3451.7175, 3445.0266,
       3421.2693, 3397.7705, 3383.8672, 3378.5867, 3376.9436, 3384.497 ,
       3397.8062, 3413.6506, 3425.5842, 3419.2698, 3396.655 , 3388.575 ,
       3399.0022, 3406.778 , 3404.4407, 3412.1252, 3453.3027, 3498.5874,
       3522.4956, 3540.7886, 3562.929 , 3580.8875, 3590.6653, 3603.5688,
       3611.049 , 3599.2512, 3588.9226, 3598.576 , 3598.2751, 3597.6016,
       3611.845 , 3617.065 , 3621.992 , 3613.4094, 3577.7725, 3552.9014,
       3537.0251, 3519.8423, 3532.2253, 3548.1487, 3542.533 , 3541.8823,
       3556.26  , 3590.2063, 3663.453 , 3715.3564, 

# BPNN

In [4]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,1])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-16.678080750825398, p_value=1.5515641881533605e-51)
crit：MAE， mdm_return(MDM=-25.211471886736682, p_value=5.231093006963664e-96)
crit：MAPE， mdm_return(MDM=-25.561256109000986, p_value=7.292485913095349e-98)


# SVR

In [5]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,2])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-16.495426453639343, p_value=1.242260006922966e-50)
crit：MAE， mdm_return(MDM=-24.623885790741326, p_value=6.883173229202305e-93)
crit：MAPE， mdm_return(MDM=-24.731405685239725, p_value=1.8488587787448376e-93)


# GRU

In [6]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,3])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-16.215927437651885, p_value=2.9481802329192834e-49)
crit：MAE， mdm_return(MDM=-24.787631512230302, p_value=9.297877300392092e-94)
crit：MAPE， mdm_return(MDM=-24.683172274451042, p_value=3.334251250544746e-93)


# LSTM

In [7]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,4])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-14.11307664600834, p_value=3.143795920617924e-39)
crit：MAE， mdm_return(MDM=-21.699209005543935, p_value=2.1813476084116858e-77)
crit：MAPE， mdm_return(MDM=-21.517404637477103, p_value=1.9834011392431045e-76)


# Transformer

In [8]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,5])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-16.835208680853032, p_value=2.5748527613959343e-52)
crit：MAE， mdm_return(MDM=-24.204618557504435, p_value=1.1590492850170353e-90)
crit：MAPE， mdm_return(MDM=-24.73300314613748, p_value=1.813101752653758e-93)


# Informer

In [9]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,6])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-11.96808910840573, p_value=9.855909627567329e-30)
crit：MAE， mdm_return(MDM=-20.00925817123301, p_value=1.6033191269456726e-68)
crit：MAPE， mdm_return(MDM=-19.758396055037895, p_value=3.240694401980225e-67)


# TCN

In [10]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,7])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-16.920535642083752, p_value=9.685035345009253e-53)
crit：MAE， mdm_return(MDM=-24.33140179791594, p_value=2.4596463378727184e-91)
crit：MAPE， mdm_return(MDM=-24.53717111246761, p_value=1.9871416555688288e-92)


# BiLSTM

In [11]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,8])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-14.81050686681985, p_value=1.7376402085367444e-42)
crit：MAE， mdm_return(MDM=-22.518705073337763, p_value=1.0165459341618105e-81)
crit：MAPE， mdm_return(MDM=-22.330422149476657, p_value=1.0083897740674124e-80)


# CNN-BiLSTM-SAM

In [12]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,9])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-11.457725308741521, p_value=1.3153452311787198e-27)
crit：MAE， mdm_return(MDM=-19.32229922664887, p_value=5.917803032907359e-65)
crit：MAPE， mdm_return(MDM=-18.901691421461496, p_value=8.760816063092519e-63)


# ICEEMDAN-SVR

In [13]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,10])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-17.87554651896977, p_value=1.5331881165851168e-57)
crit：MAE， mdm_return(MDM=-25.69567475478869, p_value=1.4127140936900694e-98)
crit：MAPE， mdm_return(MDM=-24.641794873264473, p_value=5.529638510671899e-93)


# CEEMDAN-LSTM

In [14]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,11])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-20.694637998120854, p_value=4.1937712434569457e-72)
crit：MAE， mdm_return(MDM=-26.83803024845988, p_value=1.2649390153647908e-104)
crit：MAPE， mdm_return(MDM=-27.301784467375505, p_value=4.501588132581214e-107)


# ICEEMDAN-BiGRU

In [15]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,12])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-16.15986198017687, p_value=5.550975984133457e-49)
crit：MAE， mdm_return(MDM=-18.97256529000114, p_value=3.781130474264425e-63)
crit：MAPE， mdm_return(MDM=-19.099610360948848, p_value=8.368700698798053e-64)


# VMD-BiLSTM

In [16]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,13])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-15.782327454319708, p_value=3.848056695272359e-47)
crit：MAE， mdm_return(MDM=-21.70785913408255, p_value=1.963758296369724e-77)
crit：MAPE， mdm_return(MDM=-22.107544350885348, p_value=1.521588263875619e-79)


# BiLSTM-SAM-TCN

In [17]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,14])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-11.448308040500937, p_value=1.4378679651506864e-27)
crit：MAE， mdm_return(MDM=-17.478050574525188, p_value=1.5626448208788795e-55)
crit：MAPE， mdm_return(MDM=-17.292836189016775, p_value=1.332833322525137e-54)


# ICEEMDAN-BiLSTM-SAM-TCN

In [18]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,15])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-11.388197355475047, p_value=2.5359493280227268e-27)
crit：MAE， mdm_return(MDM=-11.586042808537929, p_value=3.8907714598730665e-28)
crit：MAPE， mdm_return(MDM=-11.570758182587642, p_value=4.500285280543054e-28)


# ICEEMDAN-VMD(PSO)-SAM-TCN

In [19]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,16])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-11.541185366911664, p_value=5.96185731276585e-28)
crit：MAE， mdm_return(MDM=-16.529054409758498, p_value=8.475261799468847e-51)
crit：MAPE， mdm_return(MDM=-16.43776473535125, p_value=2.391471580914871e-50)


# ICEEMDAN-VMD(PSO)-BiLSTM-SAM

In [20]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,17])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-14.30333786127682, p_value=4.13237412323758e-40)
crit：MAE， mdm_return(MDM=-18.242847166567913, p_value=2.080056496347416e-59)
crit：MAPE， mdm_return(MDM=-18.39609731498202, p_value=3.433528574888476e-60)


# ICEEMDAN-VMD(PSO)-BiLSTM-TCN

In [21]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,18])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-20.666236489734864, p_value=5.908308828738852e-72)
crit：MAE， mdm_return(MDM=-23.355257271551626, p_value=3.740053805447281e-86)
crit：MAPE， mdm_return(MDM=-22.99509118499962, p_value=3.0417792627856223e-84)


# CEEMDAN-VMD(PSO)-BiLSTM-SAM-TCN

In [22]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,19])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-7.635507881061379, p_value=8.989238275613688e-14)
crit：MAE， mdm_return(MDM=-7.927768593794644, p_value=1.1036505055563793e-14)
crit：MAPE， mdm_return(MDM=-7.9335110671713105, p_value=1.0584391990631337e-14)


# ICEEMDAN-VMD(PSO)-BiLSTM-SAM-CNN

In [23]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,20])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-9.644402075054675, p_value=1.4989029628928612e-20)
crit：MAE， mdm_return(MDM=-9.826399586377653, p_value=3.19907227657678e-21)
crit：MAPE， mdm_return(MDM=-9.586757806473154, p_value=2.4342615089609808e-20)


# ICEEMDAN-VMD(PSO)-LSTM-SAM-TCN

In [24]:
pred2_lst  =read_csv('./True value of SSEC High and all model predictions.csv', index_col=0,usecols=[0,21])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-16.123208358486803, p_value=8.391445490836053e-49)
crit：MAE， mdm_return(MDM=-17.68303242704552, p_value=1.4452950609774944e-56)
crit：MAPE， mdm_return(MDM=-17.659296988208403, p_value=1.9048725443419045e-56)
